In [ ]:
import polars as pl
import polars_ds as pds
import numpy as np
import pocache

import logging
logging.basicConfig(level=logging.INFO)

ss = pocache.Session(mode = "mem", verbose=True, secure=False)

In [ ]:
df = pds.frame(size = 100).with_columns(
    a = pds.random()
    , b = pds.random()
    , c = pds.random_str(min_size = 1, max_size = 3)
)
df.head()

In [ ]:
from functools import lru_cache

@ss.pocache
def expensive_df_transform1(df: pl.LazyFrame, threshold: float) -> pl.DataFrame:
    import time
    time.sleep(2)

    return df.lazy().filter(
        (pl.col("c").str.len_chars() == 1)
        & (pl.col("a") < threshold)
    ).with_columns(
        deciles = pl.col("a").qcut(100, labels = [f"decile_{str(i+1).zfill(3)}" for i in range(100)])
        , d = (pl.col("a") + pl.col("b")) / 2
    ).group_by("deciles").agg(
        pl.len().alias("cnt")
        , pl.col("a").min().alias("a_min")
        , pl.col("a").max().alias("a_max")
        , pl.col("d").std().alias("d_std")
        , pl.col("b").sum().alias("b_sum")
        , pl.col("c").n_unique().alias("c_n_unique")
    ).sort("deciles").collect()

In [ ]:
expensive_df_transform1(df, threshold=0.8)

In [ ]:
expensive_df_transform1(df, threshold = 0.8)